In [ ]:
pip install aiohttp

In [2]:
pip install nest-asyncio

In [5]:
import pandas as pd
import requests
import aiohttp
import asyncio
import json
import os
import sys
import math
from aiohttp import ClientSession
from urllib.error import HTTPError
from requests.structures import CaseInsensitiveDict
import nest_asyncio
nest_asyncio.apply()

In [6]:
API = "https://api.elections.org.za"
token = 'Bi3pBe230IzZzJpc32DhARURHEBTnlKvXWs0zJLaITc9aSyZqSidkIWUTjivv0XIVg0yBKusquPqqohmgzAV11WJ8RDBWaEowPiAbJa-66zL7Fw3H95ANqgR2STR8Js9OeyB465T-PG3aDfHYb8dRBmuw1VtJi5CvY1BETCdMB0F0Rdc278_NzSbJPJgy9C2SXuOc_mrWFG6fQ-Q-gcWuzVLsRp9VHM-KqkLGA-QmdAfbEt_Kn9bRiPrZ7QC1ewpcRNazu3SD0sFLcFsMLeelC9Z6LUE4T3EP1ElvFJqWhjXNx-Oan3E5U4O2PJtyr57gQC_GIusiwWYrDOzb4qG1RXA3XC6gYr531QKGjXUETrMXNPWzLYxcuWYmAOE8Ca0ON8L0M24n5UQP4RWs7260bGdUrB-sncp9ZI9AB9pl9-O0Hm0'
headers = CaseInsensitiveDict()
headers["Authorization"] = "Bearer " + token
headers["Accept"] = "application/json"

In [14]:
ElectoralEventID = '402'
WardResults = []
MuniSeatResults = []
Result =[]


In [8]:
wards_df = pd.read_csv('Wards.csv')
Wards = wards_df.values.tolist()
munis_df = pd.read_csv('Munis.csv')
Munis = munis_df.values.tolist()

In [9]:
#remove the district 
munis_df = munis_df[munis_df['MunicTypeID']!=3]
munis_id =munis_df['MunicipalityID'].tolist()


In [10]:
async def get_data(endpoint, url, query, session):
    api_url = API + url + str(query)
    try:
        response = await session.request(method='GET', url=api_url, headers=headers)
        response.raise_for_status()
    except HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"An error ocurred: {err}")
    response_json = await response.json()
    return response_json

In [11]:

async def run_program(endpoint, url, query, session):
    
    try:
        response = await get_data(endpoint, url, query, session)
        row = dict({
             'Province': response['Province'],
             'Geography': response['Municipality'].split(" ", 1)[0],
             'MunicipalityID': response['MunicipalityID'],
             'TotalSeatsAvailable': response['TotalSeatsAvailable'],
             'TotalValidVotes':response['TotalValidVotes']
        })
        ##### 
        ## LGESeatCalculationResults-Hung
        #####
                
        if(endpoint == 'LGESeatCalculationResults-Hung'):      
            muniResults = []
             
            for result in response['PartyResults']:
                row['PartyName'] =  result['Name']
                row['TotalPartySeats'] = result['TotalPartySeats'] 
                muniResults.append(
                     {
                        'PartyName': result['Name'],
                        'TotalPartySeats': result['TotalPartySeats']
                     }
                 )
                Result.append(row) 
            
            if(response['TotalSeatsAvailable']/2 > max(muniResults, key=lambda x:x['TotalPartySeats'])['TotalPartySeats']):
                majority_result = 'Hung'
            else:
                majority_result = 'Outright Majority'
                
            MuniSeatResults.append(
                {
                    'Province': response['Province'],
                    'Geography': response['Municipality'].split(" ", 1)[0],
                    'majority_result': majority_result
                    
                }
            )

            
    except Exception as err:
        print(f"Exception occured: {err}")
        pass

In [15]:
async def getOutrightMajority():
  async with ClientSession() as session:
      await asyncio.gather(*[run_program('LGESeatCalculationResults-Hung','/api/v1/LGESeatCalculationResults?ElectoralEventID=' + str(ElectoralEventID), '&MunicipalityID=' + str(m), session) for m in munis_id])

      muniResults_df = pd.DataFrame(MuniSeatResults)
      muniResults_df.to_csv('result_2016_number_of_majority.csv', index=False)
     

def main():
    loop = asyncio.get_event_loop()
    loop.run_until_complete(getOutrightMajority())
    #loop.close()

main()

Exception occured: 0, message='Attempt to decode JSON with unexpected mimetype: ', url=URL('https://api.elections.org.za/api/v1/LGESeatCalculationResults?ElectoralEventID=402&MunicipalityID=6614')


In [16]:
muniResults_df = pd.DataFrame(MuniSeatResults)
work_df = muniResults_df
work_df2 = work_df.groupby('Province')['majority_result'].value_counts().reset_index(name='count')
work_df2
work_df2.to_csv('MunisHungByProvince.csv', index=False)

In [ ]:
df = pd.DataFrame(Result)
df.to_csv('result_2016_number_of_majority.csv', index=False)
df